In [1]:
import numpy as np
import matplotlib.pyplot as plt
import transformers
import datasets
import torch

/data/phillip_guo/miniconda3/envs/hp-unlrn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# download this huggingface model https://huggingface.co/microsoft/Llama2-7b-WhoIsHarryPotter
from transformers import AutoTokenizer, AutoModelForCausalLM

# might need to adapt to quantize for 24gb 3090, or remove .cuda()
hp_model = AutoModelForCausalLM.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter").cuda()
tokenizer = AutoTokenizer.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# remove start of sentence token
tokenizer.

In [5]:
torch.cuda.max_memory_allocated()

27087896576

In [13]:
tokenizer.encode(" Harry P")

[1, 29871, 10686, 349]

In [9]:
from tasks import IOITask, OWTTask, SportsTask
ioi_task = IOITask(batch_size=16, tokenizer=tokenizer, handle_multitoken_labels=True)


ValueError: '▁' is not in list